# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

дополнения: *Ян Пиле, НИУ ВШЭ*

Посмотрим на другие примеры использования `selenium`. 

**Пример.** Зайдем на сайт книжного магазина и найдем все книги про Python. Загрузим библиотеку, веб-драйвер и откроем страницу в браузере через Python.

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

br = webdriver.Chrome(ChromeDriverManager().install())

# открываем страницу в Chrome в автоматическом режиме
br.get("http://www.biblio-globus.ru/")

Найдем с помощью CSS Selector'а (*SelectorGadget*) поле для ввода названия книги или автора. 

In [ ]:
field = br.find_element_by_css_selector("input")

Сохраним запрос:

In [ ]:
author = "Python"  # переменная author - условность

Введем запрос в поле для поиска (`.send_keys`) и подождем чуть-чуть:

In [ ]:
field.send_keys(author)
br.implicitly_wait(2)  # подождем пару секунд

Теперь найдем кнопку для поиска (значок *лупа* рядом со строкой поиска) через CSS Selector:

In [ ]:
submit = br.find_element_by_css_selector("#search_submit")

Кликнем на нее:

In [ ]:
submit.click()

Сохраним первую страницу с результатами в переменную `page1`.

In [ ]:
page1 = br.page_source

In [ ]:
page1

Теперь обработаем эту страницу через `BeautifulSoup`:

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup1 = BeautifulSoup(page1, 'lxml')

Найдем все названия книг на этой странице. По исходному коду можно увидеть, что они имеют тэг `a` с атрибутом `class`, равным `name`:

In [ ]:
soup1.find_all('a', {'class':'name'})

С помощью списковых включений выберем из ссылок с тэгом `<a>` текст (так мы уже делали, и не раз).

In [ ]:
books1 = [b.text for b in soup1.find_all('a', {'class':'name'})]

In [ ]:
books1

Теперь аналогичным образом сгрузим информацию об авторах:

In [ ]:
# [a.text for a in soup1.find_all('div', {'class': 'author'})]
# то же самое что и
authors1 = []
for a in soup1.find_all('div', {'class': 'author'}):
    authors1.append(a.text)

In [ ]:
authors1

In [ ]:
soup1.find_all('div', {'class': 'author'})

In [ ]:
authors1 = [a.text for a in soup1.find_all('div', {'class': 'author'})]

Сгрузим расположение:

In [ ]:
#place1 = [p.text for p in soup1.find_all('div', {'class':'placement'})]

place1 = []
for p in soup1.find_all('div', {'class': 'details_1'}):
    if 'Расположение' in p.text:
        place1.append(p.find('div', {'class': 'placement'}).text)
        #place1 = p.find('div', {'class': 'placement'}).append(p.text)
    else:
        place1.append(None)

In [ ]:
place1

И, конечно, цену:

In [ ]:
price1 = [p.text for p in soup1.find_all('div', 
                                         {'class':'title_data price'})]

In [ ]:
price1

Осталось пройтись по всем страницам, которые были выданы в результате поиска. Для примера перейдем на страницу 2 и на этом остановимся.

In [ ]:
next_p = br.find_element_by_css_selector('.next_page')

In [ ]:
next_p.click()

Проделаем то же самое, что и с первой страницей. По-хорошему нужно написать функцию, которая будет искать на странице названия книг, их расположение и цену. Но оставим это в качестве задания читателю :)

In [ ]:
page2 = br.page_source
soup2 = BeautifulSoup(page2, 'lxml')
books2 = [b.text for b in soup2.find_all('a', {'class':'name'})]
author2 = [a.text for a in soup2.find_all('div', {'class': 'author'})]
place2 = [p.text for p in soup2.find_all('div', {'class':'placement'})]
price2 = [p.text for p in soup2.find_all('div', {'class':'title_data price'})]

Расширим списки результатов с первой страницы данными, полученными со второй страницы, используя метод `.extend()`.

In [ ]:
books1.extend(books2) # books1 + books2
authors1.extend(books2)
place1.extend(place2)
price1.extend(price2)

Осталось импортировать библиотеку `pandas` и создать датафрейм.

In [ ]:
import pandas as pd

In [ ]:
len(books1)

In [ ]:
len(authors1)

In [ ]:
len(place1)

In [ ]:
len(price1)

Для разнообразия создадим датафрейм не из списка списков, а из словаря. Ключами словаря будут названия столбцов в таблице, а значениями – списки с сохраненной информацией (названия книг, цены и проч.).

In [ ]:
df = pd.DataFrame({'book': books1, 'author': authors1,
                   'placement': place1, 'price': price1})

In [ ]:
df.head(10)

In [ ]:
df.info()

Давайте приведем столбец с ценой к числовому типу. Уберем слова *Цена* и *руб*, а потом сконвертируем строки в числа с плавающей точкой. Напишем функцию `get_price()`,

In [ ]:
df.iloc[1, 3]

In [ ]:
float(df.iloc[1, 3].split()[1].replace(',', '.'))

In [ ]:
import re

In [ ]:
float('.'.join(re.findall(r'\d+', df.iloc[1, 3])))

In [ ]:
re.findall(r'\d+', df.iloc[1, 3])

In [ ]:
float(df.iloc[1, 3].split()[1].replace(',', '.'))

In [ ]:
def get_price(price):
    book_price = price.split(' ')[1]  # разобьем строку по пробелу и возьмем второй элемент
    book_price = book_price.replace(',', '.')  # заменим запятую на точку
    price_num = float(book_price)  # сконвертируем в float
    return price_num

In [ ]:
def price(x):
    return float('.'.join(re.findall(r'\d+',x)))

In [ ]:
# проверка
get_price(df.price[0])

In [ ]:
price(df.price[0])

Всё отлично работает! Применим функцию к столбцу *price* и создадим новый столбец *nprice*.

In [ ]:
df['nprice'] = df.price.apply(price)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Теперь можем расположить книги по цене в порядке возрастания:

In [ ]:
df.sort_values('nprice')

И сохраним всю таблицу в csv-файл:

In [ ]:
df.to_csv("books.csv", index=False)

In [ ]:
br.close()

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

br = webdriver.Chrome(ChromeDriverManager().install())

# открываем страницу в Chrome в автоматическом режиме
br.get("http://www.biblio-globus.ru/")

In [ ]:
books = '//*[@id="TableMRight"]/tbody/tr/td/table/tbody/tr[2]/td[2]/a'

In [ ]:
books_el = br.find_element_by_xpath(books)

In [ ]:
books_el.click()

In [ ]:
det_css = 'body > table > tbody > tr:nth-child(2) > td.column_right > div > div.card-columns > div:nth-child(1) > div > ul > li > ul > li:nth-child(2) > ul > li:nth-child(2) > a'

In [ ]:
det_el = br.find_element_by_css_selector(det_css)

In [ ]:
det_el.click()

In [ ]:
page1 = BeautifulSoup(br.page_source)

In [ ]:
books_p1 = page1.find_all('div', {'class': 'details_1'})

In [ ]:
len(books_p1)

In [ ]:
books_p1[0]

In [ ]:
books_p1[0].div is not None

In [ ]:
books_p1[0].find('div', {'class': 'author'}).text

In [ ]:
books_p1[0].a.text

In [ ]:
books_p1[0].find('div', {'class': 'placement'}).text

In [ ]:
books_p1[0].find('div', {'class': 'title_data price'}).text

In [ ]:
books_p1[0].find('div', {'class': 'title_data pricee'}) is None

In [ ]:
titles = []
authors = []
places = []
prices = []
for book in books_p1:
    if book.find('div', {'class': 'author'}) is not None:
        authors.append(book.find('div', {'class': 'author'}).text)
    else:
        authors.append('')
        
    if book.a is not None:
        titles.append(book.a.text)
    else:
        titles.append('')
        
    if book.find('div', {'class': 'placement'}) is not None:
        places.append(book.find('div', {'class': 'placement'}).text)
    else:
        places.append('')
        
    if book.find('div', {'class': 'title_data price'}) is not None:
        prices.append(book.find('div', {'class': 'title_data price'}).text)
    else:
        prices.append('')

In [ ]:
titles

In [ ]:
def get_page_info(books_p):
    titles = []
    authors = []
    places = []
    prices = []
    for book in books_p:
        if book.div is not None:
            authors.append(book.div.text)
        else:
            authors.append('')

        if book.a is not None:
            titles.append(book.a.text)
        else:
            titles.append('')

        if book.find('div', {'class': 'placement'}) is not None:
            places.append(book.find('div', {'class': 'placement'}).text)
        else:
            places.append('')

        if book.find('div', {'class': 'title_data price'}) is not None:
            prices.append(book.find('div', {'class': 'title_data price'}).text)
        else:
            prices.append('')
            
    return titles, authors, places, prices

In [ ]:
next_page = '//*[@id="main_wrapper"]/ul/li[4]/a'

In [ ]:
page2 = br.find_element_by_xpath(next_page)

In [ ]:
page2.click()

In [ ]:
# next_page_2 = '//*[@id="main_wrapper"]/ul/li[8]/a'
# next_page_2 = br.find_element_by_xpath(next_page_2)
# next_page_2.click()

In [ ]:
from time import sleep

In [ ]:
np_xpath = '//*[@id="main_wrapper"]/ul/li[8]/a/span[1]'

In [ ]:
for _ in range(3):
    try:
        page = BeautifulSoup(br.page_source)
        books = page.find_all('div', {'class': 'details_1'})
        t, a, pl, pr = get_page_info(books)
        titles.extend(t)
        authors.extend(a)
        places.extend(pl)
        prices.extend(pr)
        np = br.find_element_by_xpath(np_xpath)
        sleep(3)
        np.click()
    except:
        print('all pages parsed')
        break

In [ ]:
df = pd.DataFrame({'book': titles, 'author': authors,
                   'placement': places, 'price': prices})

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
br.close()